In [ ]:
import os
import cv2 as cv
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

In [ ]:
#This function loads images from a specific folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        if img is not None:
            images.append(img)
    return images

In [ ]:
def build_graph(imgs, adj_matrix, weight_matrix):
    G=nx.Graph()
    n=len(imgs)
    for i in range(0,n):
        G.add_node(i,image = imgs[i])
    for i in range(n): 
         for j in range(n): 
            if adj_matrix[i,j] == 1:
                G.add_edge(i,j, weight=max(weight_matrix[i,j],weight_matrix[j,i])) 
    return G

In [ ]:
def advanced_print_graph(imgs, G, save_images = True, graph_dir = "output", graph_name = "graph"):
    
    pos=nx.circular_layout(G)
    fig=plt.figure(figsize=(15,15))
    ax=plt.subplot(111)
    ax.set_aspect('equal')
    nx.draw(G,pos,ax=ax, width = 3, node_size=900,with_labels = True, 
            edgecolors='red', node_color='lightgray', connectionstyle='arc3, rad = 0.1')
    
    labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
    plt.xlim(-1.5,1.5)
    plt.ylim(-1.5,1.5)
    trans=ax.transData.transform
    trans2=fig.transFigure.inverted().transform
    piesize=0.08 # this is the image size
    p2=piesize/0.9 #this is the image center
    for g in G:
        xx,yy=trans(pos[g]) # figure coordinates
        xa,ya=trans2((xx,yy)) # axes coordinates
        a = plt.axes([xa-p2,ya-p2, piesize, piesize])
        a.set_aspect('equal')
        a.imshow(G.nodes[g]['image'])
        a.axis('off')
    ax.axis('off')
    if save_images:
        plt.savefig(os.path.join(graph_dir,f'{graph_name}.png'))
    return G

In [ ]:
def build_and_print_multi_graph(adj_matrix, save_images = True, graph_dir = "output", graph_name = "graph"):
    graph = nx.from_numpy_matrix(adj_matrix, parallel_edges=True, create_using=nx.MultiDiGraph)
    pos=nx.circular_layout(graph)
    fig = plt.figure(figsize=(10,10))
    nx.draw(graph, pos=pos, with_labels=True, connectionstyle='arc3, rad = 0.1')
    plt.show()
    if save_images:
        fig.savefig(os.path.join(graph_dir,f'{graph_name}.png'))
    return graph

In [ ]:
def print_graph(graph, save_images = True, graph_dir = "output", graph_name = "graph"):
    pos=nx.spring_layout(graph)
    fig=plt.figure(figsize=(10,10))
    ax=plt.subplot(111)
    ax.set_aspect('equal')
    nx.draw(graph,pos,ax=ax, width = 3, node_size=900,with_labels = True, edgecolors='red', node_color='lightgray')
    plt.show()
    if save_images:
        fig.savefig(os.path.join(graph_dir,f'{graph_name}.png'))

In [ ]:
def get_homographies_from_states(U, idx_ref=0):
    #Compute the homography of each image w.r.t. the reference one
    H = [np.dot(U[idx_ref],np.linalg.inv(U[i])) for i in range(len(U))]
    return H

In [ ]:
def get_reference_node(adj_matrix):
    return np.argmax(np.sum(adj_matrix, axis=0))

In [ ]:
#This function splits a 3nx3n matrix into a set of 3x3 matrices
def split_states(x):
    x_small = x.transpose()
    res = [ x_small[:,i*3:(i+1)*3].transpose() for i in range(x_small.shape[1]//3)]
    return res

In [ ]:
def get_normalization_matrix(imgs):
    RESCALE = 1./np.max(imgs[0].shape)
    T_norm = np.diag([RESCALE,RESCALE,1])
    return T_norm